In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

plt.rcParams["font.sans-serif"] = ["SimHei"]  #设置字体
plt.rcParams["axes.unicode_minus"] = False  #该语句解决图像中的“-”负号的乱码问题
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import os

os.chdir(r'D:\pycharm-python\datasets')
import seaborn as sns

sns.set_theme(style='whitegrid')
from sklearn.model_selection import KFold, cross_val_score, cross_validate, GridSearchCV
from sklearn import metrics
import scipy

In [24]:
def load_data():
    df = pd.read_csv(r'0219_train_data.csv')
    df = df.drop(['Unnamed: 0'], axis=1)
    new_df = df.copy()
    arr = np.array(new_df)[:, 2:]
    x = arr.copy()
    # y = arr[:, -1:].copy()
    return df, x

In [25]:
df,x = load_data()

In [26]:
df1 = df.drop(['Province','City'],axis=1)
df1

,GDP,PopDenity,Urban_rate,Cars,Elec,DN
0,362.38,23222.00,93.18,4.1205,5.50,310.49
1,286.02,1337.00,50.94,4.8136,3.27,31.47
2,131.03,1364.00,44.23,9.9600,12.52,18.16
3,68.37,66.00,30.54,0.9496,1.68,7.15
4,1404.70,2045.00,85.87,106.0000,335.90,2147.46
...,...,...,...,...,...,...
118,53.21,205.62,35.21,2.5500,12.20,15.74
119,1125.46,1462.73,59.22,124.5900,344.83,856.36
120,294.22,365.68,41.18,30.3800,107.92,236.13
121,180.78,192.17,38.58,20.7900,124.70,104.23


1. 第一步，统筹指标方向。由于我们全部都是正向指标，所以不需要这一步。

In [27]:
for column in df1.columns:
    df1[column] = df1[column].apply(
        lambda x:x/np.sum(df1[column])
    )

In [28]:
df1.head()

,GDP,PopDenity,Urban_rate,Cars,Elec,DN
0,0.003581,0.108461,0.013766,0.000414,0.000505,0.006777
1,0.002826,0.006245,0.007525,0.000484,0.000300,0.000687
2,0.001295,0.006371,0.006534,0.001000,0.001149,0.000396
3,0.000676,0.000308,0.004512,0.000095,0.000154,0.000156
4,0.013881,0.009551,0.012686,0.010647,0.030821,0.046871


In [29]:
df1.iloc[:,1].head()

0    0.108461
1    0.006245
2    0.006371
3    0.000308
4    0.009551
Name: PopDenity, dtype: float64

In [30]:
# 计算信息熵
K = 1/ np.log(df1.shape[0])
Entroy = []
for i in range(df1.shape[1]):
    e_j = -K * np.sum(df1.iloc[:,i]*np.log(df1.iloc[:,i]))
    Entroy.append(e_j)

In [31]:
Entroy

[0.7744669746960875,
 0.5959345792054515,
 0.9897880784094099,
 0.8781225706201302,
 0.8136135893684291,
 0.8519041185797136]

In [32]:
D_j = [1-i for i in Entroy]
sum = sum(D_j)
W_j = [i/sum for i in D_j]

TypeError: 'numpy.float64' object is not callable

In [33]:
 W_j

[0.2057463778133275,
 0.3686156234372746,
 0.009316000949068932,
 0.11118477925047733,
 0.1700342058968866,
 0.13510301265296512]

### 结合TOPSIS进行加权

1. 指标同向化，由于我们所有的指标都是正向指标。所以我们不需要进行同质化。
2. 规范化矩阵。


In [34]:
column = df1.columns[1]
column
df1[column].head()

0    0.108461
1    0.006245
2    0.006371
3    0.000308
4    0.009551
Name: PopDenity, dtype: float64

In [35]:
maxs = []
mins = []
for column in df1.columns:
    maxs.append(np.max(df1[column]))
    mins.append(np.min(df1[column]))

In [36]:
df1.shape
# max_array = np.vstack(maxs,)
df1.iloc[1,2]

0.007525494940899603

In [37]:
D_plus = []
D_minus = []
for i in range(df1.shape[0]):
    D_sum = 0
    d_sum = 0
    for j in range(df1.shape[-1]):
        D_sum+=W_j[j]*(df1.iloc[i,j]-maxs[j])**2
        d_sum+=W_j[j]*(df1.iloc[i,j]-mins[j])**2
    D_plus.append(np.sqrt(D_sum))
    D_minus.append(np.sqrt(d_sum))
